<a href="https://colab.research.google.com/github/packetech/baracuda/blob/master/GreatLearning%20World-wide%20hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size=6> Building a model to predict Great Stone's mutual funds ratings, to help investors make informed decisions </font>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install sklearn

In [ ]:
# load libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from scipy.stats import zscore
from sklearn import metrics

from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline


import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization


from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping


#### Data pre-processing

In [ ]:
# Reading this data and dropping un-necessary columns

dfbond_ratings = pd.read_csv('/content/drive/My Drive/AIML/Hackathon/bond_ratings.csv')
dfbond_ratings.head(2)

,bb_rating,us_govt_bond_rating,below_b_rating,others_rating,maturity_bond,b_rating,tag,a_rating,aaa_rating,aa_rating,bbb_rating,duration_bond
0,0.0,0.0,0.0,0.0,NaN,0.0,67922,0.0,0.0,0.0,0.0,NaN
1,0.0,0.0,0.0,0.0,NaN,0.0,134783,0.0,0.0,0.0,0.0,NaN


In [ ]:
dfbond_ratings.describe().T

,count,mean,std,min,25%,50%,75%,max
bb_rating,24886.0,3.473804,8.299747,0.00,0.00,0.00,2.450,80.47
us_govt_bond_rating,24886.0,0.000000,0.000000,0.00,0.00,0.00,0.000,0.00
below_b_rating,24886.0,0.827525,2.700019,-0.02,0.00,0.00,0.100,42.30
others_rating,24886.0,1.666769,6.885158,-68.21,0.00,0.00,0.330,100.00
maturity_bond,8093.0,7.765444,4.148558,0.00,5.46,7.29,8.920,29.30
b_rating,24886.0,3.234399,9.197164,-0.12,0.00,0.00,0.710,80.68
tag,25000.0,139878.934640,65731.287074,26000.00,83021.75,139881.50,196759.250,253763.00
a_rating,24886.0,5.054436,9.261811,0.00,0.00,0.00,8.340,72.87
aaa_rating,24886.0,14.558310,25.636838,-3.15,0.00,0.00,18.955,118.65
aa_rating,24886.0,4.209073,11.164511,-0.19,0.00,0.00,3.010,90.22


In [ ]:
dfbond_ratings = dfbond_ratings.drop(labels="us_govt_bond_rating", axis = 1) # all zeros

In [ ]:
# Reading this data

dffund_allocations = pd.read_csv('/content/drive/My Drive/AIML/Hackathon/fund_allocations.csv')
dffund_allocations.head(2)

,portfolio_communication_allocation,portfolio_financial_services,portfolio_industrials_allocation,portfolio_tech_allocation,portfolio_materials_basic_allocation,portfolio_energy_allocation,portfolio_consumer_defence_allocation,portfolio_healthcare_allocation,portfolio_property_allocation,id,portfolio_utils_allocation,portfolio_cyclical_consumer_allocation
0,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00,0.00,67922,0.00,0.00
1,0.78,9.77,9.97,35.51,2.86,0.38,5.88,14.41,2.67,134783,0.39,17.38


In [ ]:
dffund_allocations = dffund_allocations.rename(columns = {"id": "tag"}) # Renaming to aid merging

In [ ]:
# Reading this data

dffund_ratios = pd.read_csv('/content/drive/My Drive/AIML/Hackathon/fund_ratios.csv')
dffund_ratios.head(2)

,fund_id,tag,fund_ratio_net_annual_expense,pb_ratio,ps_ratio,mmc,pc_ratio,pe_ratio
0,264614c6-5ac3-4146-ba26-1674b136cb40,67922,1.44,1.71,1.31,"19,857.41",5.91,14.51
1,f5ad58c2-fdea-4087-8678-e04744f89f90,134783,0.58,5.30,3.38,"72,347.03",15.95,18.88


In [ ]:
# Reading this data and dropping un-necessary columns

dffund_specs = pd.read_csv('/content/drive/My Drive/AIML/Hackathon/fund_specs.csv')
dffund_specs.head(2)

,investment_class,currency,total_assets,yield,greatstone_rating,inception_date,tag,fund_size,return_ytd
0,Value,USD,1.185000e+07,5.57,NaN,2015-02-02,67922,Large,20.19
1,Growth,USD,1.397000e+10,0.42,3.0,2012-05-30,134783,Large,16.79


In [ ]:
dffund_specs = dffund_specs.drop(labels="currency", axis = 1) # It makes no sense 

In [ ]:
# Reading this data

dfother_specs = pd.read_csv('/content/drive/My Drive/AIML/Hackathon/other_specs.csv')
dfother_specs.head(2)

,2014_category_return,2012_return_category,years_up,2018_return_category,tag,category_return_1year,cash_percent_of_portfolio,pc_ratio,2011_return_category,ytd_return_fund,years_down,2014_return_fund,category_return_1month,2013_return_fund,fund_return_3months,ytd_return_category,pb_ratio,2017_category_return,1_year_return_fund,pe_ratio,2015_return_fund,portfolio_convertable,3_months_return_category,portfolio_others,2016_return_fund,mmc,stock_percent_of_portfolio,2016_return_category,ps_ratio,2011_return_fund,2010_return_fund,fund_return_3years,2012_fund_return,2018_return_fund,2017_return_fund,greatstone_rating,category_ratio_net_annual_expense,category_return_2015,1_month_fund_return,bond_percentage_of_porfolio,portfolio_preferred,2010_return_category,2013_category_return
0,NaN,NaN,1.0,-16.32,67922,13.05,1.19,5.91,NaN,20.19,2.0,NaN,4.20,NaN,20.19,19.10,1.71,-5.78,18.40,14.51,NaN,0.0,19.10,0.0,16.14,"19,857.41",98.81,27.30,1.31,NaN,NaN,4.24,NaN,-12.23,-3.31,NaN,1.75,-34.98,4.12,0.0,0.0,NaN,NaN
1,10.0,15.34,5.0,-2.09,134783,10.71,0.10,15.95,NaN,16.79,1.0,14.25,2.12,35.46,16.79,15.67,5.30,27.67,12.18,18.88,5.6,0.0,15.67,0.0,1.64,"72,347.03",99.90,3.23,3.38,NaN,NaN,14.39,NaN,-2.62,26.39,3.0,1.06,3.60,2.33,0.0,0.0,NaN,33.92


In [ ]:
# Reading this data

dfreturn_3year = pd.read_csv('/content/drive/My Drive/AIML/Hackathon/return_3year.csv')
dfreturn_3year.head(2)

,tag,3yrs_treynor_ratio_fund,3_years_alpha_fund,3years_category_std,3yrs_sharpe_ratio_fund,3yrs_treynor_ratio_category,3_years_return_mean_annual_fund,fund_beta_3years,3years_fund_r_squared,3years_fund_std,category_beta_3years,fund_return_3years,3_years_alpha_category,3_years_return_mean_annual_category,3yrs_sharpe_ratio_category,3years_category_r_squared,3_years_return_category
0,67922,2.46,-7.10,0.18,0.26,0.05,0.45,1.20,54.83,16.25,0.01,4.24,-0.04,0.01,0.00,0.42,7.36
1,134783,12.2,0.07,0.13,1.06,0.13,1.19,1.07,88.46,12.26,0.01,14.39,0.01,0.01,0.01,0.84,15.35


In [ ]:
# Reading this data

dfreturn_5year = pd.read_csv('/content/drive/My Drive/AIML/Hackathon/return_5year.csv')
dfreturn_5year.head(2)

,category_r_squared_5years,5yrs_sharpe_ratio_fund,5_years_alpha_fund,5years_fund_r_squared,5years_fund_std,5yrs_sharpe_ratio_category,5_years_beta_fund,5yrs_treynor_ratio_fund,5_years_return_mean_annual_fund,5_years_return_mean_annual_category,5yrs_treynor_ratio_category,5_years_return_fund,5_years_alpha_category,5_years_beta_category,5years_category_std,tag,5_years_return_category
0,0.51,NaN,NaN,NaN,NaN,-0.00,NaN,NaN,NaN,-0.00,-0.04,0.00,-0.11,0.01,0.20,67922,-4.25
1,0.86,0.89,0.34,90.11,12.4,0.01,1.05,10.37,0.99,0.01,0.10,11.71,-0.00,0.01,0.13,134783,11.26


In [ ]:
# Reading this data

dfreturn_10year = pd.read_csv('/content/drive/My Drive/AIML/Hackathon/return_10year.csv')
dfreturn_10year.head(2)

,10years_category_r_squared,10yrs_sharpe_ratio_fund,10_years_alpha_fund,10years_fund_r_squared,10years_fund_std,10yrs_sharpe_ratio_category,10_years_beta_fund,10yrs_treynor_ratio_fund,fund_id,10_years_return_mean_annual_category,10yrs_treynor_ratio_category,10_years_return_fund,10_years_alpha_category,10_years_beta_category,10years_category_std,10_years_return_mean_annual_fund,10_years_return_category
0,0.49,NaN,NaN,NaN,NaN,0.01,NaN,NaN,264614c6-5ac3-4146-ba26-1674b136cb40,0.01,0.21,0.00,0.06,0.01,0.13,NaN,14.30
1,0.88,1.16,0.16,91.68,14.3,0.01,1.08,15.57,f5ad58c2-fdea-4087-8678-e04744f89f90,0.01,0.15,17.25,-0.01,0.01,0.14,1.42,15.94


In [ ]:

result1 = pd.merge(dffund_ratios,
                 dffund_specs,
                 on='tag', 
                 how='left')

In [ ]:
result2 = pd.merge(result1,
                 dfbond_ratings,
                 on='tag', 
                 how='left')

In [ ]:
result3 = pd.merge(result2,
                 dffund_allocations,
                 on='tag', 
                 how='left')

In [ ]:
result4 = pd.merge(result3,
                 dfother_specs,
                 on='tag', 
                 how='left')

In [ ]:
result5 = pd.merge(result4,
                 dfreturn_3year,
                 on='tag', 
                 how='left')

In [ ]:
result6 = pd.merge(result5,
                 dfreturn_5year,
                 on='tag', 
                 how='left')

In [ ]:
result7 = pd.merge(result6,
                 dfreturn_10year,
                 on='fund_id', 
                 how='left')

In [ ]:
pd.set_option('display.max_columns',130) # to see all the columns

In [ ]:
result7.head(2)

,fund_id,tag,fund_ratio_net_annual_expense,pb_ratio_x,ps_ratio_x,mmc_x,pc_ratio_x,pe_ratio_x,investment_class,total_assets,yield,greatstone_rating_x,inception_date,fund_size,return_ytd,bb_rating,below_b_rating,others_rating,maturity_bond,b_rating,a_rating,aaa_rating,aa_rating,bbb_rating,duration_bond,portfolio_communication_allocation,portfolio_financial_services,portfolio_industrials_allocation,portfolio_tech_allocation,portfolio_materials_basic_allocation,portfolio_energy_allocation,portfolio_consumer_defence_allocation,portfolio_healthcare_allocation,portfolio_property_allocation,portfolio_utils_allocation,portfolio_cyclical_consumer_allocation,2014_category_return,2012_return_category,years_up,2018_return_category,category_return_1year,cash_percent_of_portfolio,pc_ratio_y,2011_return_category,ytd_return_fund,years_down,2014_return_fund,category_return_1month,2013_return_fund,fund_return_3months,ytd_return_category,pb_ratio_y,2017_category_return,1_year_return_fund,pe_ratio_y,2015_return_fund,portfolio_convertable,3_months_return_category,portfolio_others,2016_return_fund,mmc_y,stock_percent_of_portfolio,2016_return_category,ps_ratio_y,2011_return_fund,2010_return_fund,fund_return_3years_x,2012_fund_return,2018_return_fund,2017_return_fund,greatstone_rating_y,category_ratio_net_annual_expense,category_return_2015,1_month_fund_return,bond_percentage_of_porfolio,portfolio_preferred,2010_return_category,2013_category_return,3yrs_treynor_ratio_fund,3_years_alpha_fund,3years_category_std,3yrs_sharpe_ratio_fund,3yrs_treynor_ratio_category,3_years_return_mean_annual_fund,fund_beta_3years,3years_fund_r_squared,3years_fund_std,category_beta_3years,fund_return_3years_y,3_years_alpha_category,3_years_return_mean_annual_category,3yrs_sharpe_ratio_category,3years_category_r_squared,3_years_return_category,category_r_squared_5years,5yrs_sharpe_ratio_fund,5_years_alpha_fund,5years_fund_r_squared,5years_fund_std,5yrs_sharpe_ratio_category,5_years_beta_fund,5yrs_treynor_ratio_fund,5_years_return_mean_annual_fund,5_years_return_mean_annual_category,5yrs_treynor_ratio_category,5_years_return_fund,5_years_alpha_category,5_years_beta_category,5years_category_std,5_years_return_category,10years_category_r_squared,10yrs_sharpe_ratio_fund,10_years_alpha_fund,10years_fund_r_squared,10years_fund_std,10yrs_sharpe_ratio_category,10_years_beta_fund,10yrs_treynor_ratio_fund,10_years_return_mean_annual_category,10yrs_treynor_ratio_category,10_years_return_fund,10_years_alpha_category,10_years_beta_category,10years_category_std,10_years_return_mean_annual_fund,10_years_return_category
0,264614c6-5ac3-4146-ba26-1674b136cb40,67922,1.44,1.71,1.31,"19,857.41",5.91,14.51,Value,1.185000e+07,5.57,NaN,2015-02-02,Large,20.19,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,1.0,-16.32,13.05,1.19,5.91,NaN,20.19,2.0,NaN,4.20,NaN,20.19,19.10,1.71,-5.78,18.40,14.51,NaN,0.0,19.10,0.0,16.14,"19,857.41",98.81,27.30,1.31,NaN,NaN,4.24,NaN,-12.23,-3.31,NaN,1.75,-34.98,4.12,0.0,0.0,NaN,NaN,2.46,-7.10,0.18,0.26,0.05,0.45,1.20,54.83,16.25,0.01,4.24,-0.04,0.01,0.00,0.42,7.36,0.51,NaN,NaN,NaN,NaN,-0.00,NaN,NaN,NaN,-0.00,-0.04,0.00,-0.11,0.01,0.20,-4.25,0.49,NaN,NaN,NaN,NaN,0.01,NaN,NaN,0.01,0.21,0.00,0.06,0.01,0.13,NaN,14.30
1,f5ad58c2-fdea-4087-8678-e04744f89f90,134783,0.58,5.30,3.38,"72,347.03",15.95,18.88,Growth,1.397000e+10,0.42,3.0,2012-05-30,Large,16.79,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.78,9.77,9.97,35.51,2.86,0.38,5.88,14.41,2.67,0.39,17.38,10.0,15.34,5.0,-2.09,10.71,0.10,15.95,NaN,16.79,1.0,14.25,2.12,35.46,16.79,15.67,5.30,27.67,12.18,18.88,5.6,0.0,15.67,0.0,1.64,"72,347.03",99.90,3.23,3.38,NaN,NaN,14.39,NaN,-2.62,26.39,3.0,1.06,3.60,2.33,0.0,0.0,NaN,33.92,12.2,0.07,0.13,1.06,0.13,1.19,1.07,88.46,12.26,0.01,14.39,0.01,0.01,0.01,0.84,15.35,0.86,0.89,0.34,90.11,12.4,0.01,1.05,10.37,0.99,0.01,0.10,11.71,-0.00,0.01,0.13,11.26,0.88,1.16,0.16,91.68,14.3,0.01,1.08,15.57,0.01,0.15,17.25,-0.01,0.01,0.14,1.42,15.94


In [ ]:
# removing duplicate columns

result7 = result7.drop(labels="pb_ratio_y", axis = 1)
result7 = result7.drop(labels="ps_ratio_y", axis = 1)
result7 = result7.drop(labels="mmc_y", axis = 1)
result7 = result7.drop(labels="pc_ratio_y", axis = 1)
result7 = result7.drop(labels="pe_ratio_y", axis = 1)
result7 = result7.drop(labels="greatstone_rating_y", axis = 1)
result7 = result7.drop(labels="fund_return_3years_y", axis = 1)


In [ ]:
# renaming the original columns that were duplicated and had their names changed

result7 = result7.rename(columns = {"pb_ratio_x": "pb_ratio"})
result7 = result7.rename(columns = {"ps_ratio_x": "ps_ratio"})
result7 = result7.rename(columns = {"mmc_x": "mmc"})
result7 = result7.rename(columns = {"pc_ratio_x": "pc_ratio"})
result7 = result7.rename(columns = {"pe_ratio_x": "pe_ratio"})
result7 = result7.rename(columns = {"greatstone_rating_x": "greatstone_rating"})
result7 = result7.rename(columns = {"fund_return_3years_x": "fund_return_3years"})


In [ ]:
result7.head(2)

,fund_id,tag,fund_ratio_net_annual_expense,pb_ratio,ps_ratio,mmc,pc_ratio,pe_ratio,investment_class,total_assets,yield,greatstone_rating,inception_date,fund_size,return_ytd,bb_rating,below_b_rating,others_rating,maturity_bond,b_rating,a_rating,aaa_rating,aa_rating,bbb_rating,duration_bond,portfolio_communication_allocation,portfolio_financial_services,portfolio_industrials_allocation,portfolio_tech_allocation,portfolio_materials_basic_allocation,portfolio_energy_allocation,portfolio_consumer_defence_allocation,portfolio_healthcare_allocation,portfolio_property_allocation,portfolio_utils_allocation,portfolio_cyclical_consumer_allocation,2014_category_return,2012_return_category,years_up,2018_return_category,category_return_1year,cash_percent_of_portfolio,2011_return_category,ytd_return_fund,years_down,2014_return_fund,category_return_1month,2013_return_fund,fund_return_3months,ytd_return_category,2017_category_return,1_year_return_fund,2015_return_fund,portfolio_convertable,3_months_return_category,portfolio_others,2016_return_fund,stock_percent_of_portfolio,2016_return_category,2011_return_fund,2010_return_fund,fund_return_3years,2012_fund_return,2018_return_fund,2017_return_fund,category_ratio_net_annual_expense,category_return_2015,1_month_fund_return,bond_percentage_of_porfolio,portfolio_preferred,2010_return_category,2013_category_return,3yrs_treynor_ratio_fund,3_years_alpha_fund,3years_category_std,3yrs_sharpe_ratio_fund,3yrs_treynor_ratio_category,3_years_return_mean_annual_fund,fund_beta_3years,3years_fund_r_squared,3years_fund_std,category_beta_3years,3_years_alpha_category,3_years_return_mean_annual_category,3yrs_sharpe_ratio_category,3years_category_r_squared,3_years_return_category,category_r_squared_5years,5yrs_sharpe_ratio_fund,5_years_alpha_fund,5years_fund_r_squared,5years_fund_std,5yrs_sharpe_ratio_category,5_years_beta_fund,5yrs_treynor_ratio_fund,5_years_return_mean_annual_fund,5_years_return_mean_annual_category,5yrs_treynor_ratio_category,5_years_return_fund,5_years_alpha_category,5_years_beta_category,5years_category_std,5_years_return_category,10years_category_r_squared,10yrs_sharpe_ratio_fund,10_years_alpha_fund,10years_fund_r_squared,10years_fund_std,10yrs_sharpe_ratio_category,10_years_beta_fund,10yrs_treynor_ratio_fund,10_years_return_mean_annual_category,10yrs_treynor_ratio_category,10_years_return_fund,10_years_alpha_category,10_years_beta_category,10years_category_std,10_years_return_mean_annual_fund,10_years_return_category
0,264614c6-5ac3-4146-ba26-1674b136cb40,67922,1.44,1.71,1.31,"19,857.41",5.91,14.51,Value,1.185000e+07,5.57,NaN,2015-02-02,Large,20.19,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,1.0,-16.32,13.05,1.19,NaN,20.19,2.0,NaN,4.20,NaN,20.19,19.10,-5.78,18.40,NaN,0.0,19.10,0.0,16.14,98.81,27.30,NaN,NaN,4.24,NaN,-12.23,-3.31,1.75,-34.98,4.12,0.0,0.0,NaN,NaN,2.46,-7.10,0.18,0.26,0.05,0.45,1.20,54.83,16.25,0.01,-0.04,0.01,0.00,0.42,7.36,0.51,NaN,NaN,NaN,NaN,-0.00,NaN,NaN,NaN,-0.00,-0.04,0.00,-0.11,0.01,0.20,-4.25,0.49,NaN,NaN,NaN,NaN,0.01,NaN,NaN,0.01,0.21,0.00,0.06,0.01,0.13,NaN,14.30
1,f5ad58c2-fdea-4087-8678-e04744f89f90,134783,0.58,5.30,3.38,"72,347.03",15.95,18.88,Growth,1.397000e+10,0.42,3.0,2012-05-30,Large,16.79,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.78,9.77,9.97,35.51,2.86,0.38,5.88,14.41,2.67,0.39,17.38,10.0,15.34,5.0,-2.09,10.71,0.10,NaN,16.79,1.0,14.25,2.12,35.46,16.79,15.67,27.67,12.18,5.6,0.0,15.67,0.0,1.64,99.90,3.23,NaN,NaN,14.39,NaN,-2.62,26.39,1.06,3.60,2.33,0.0,0.0,NaN,33.92,12.2,0.07,0.13,1.06,0.13,1.19,1.07,88.46,12.26,0.01,0.01,0.01,0.01,0.84,15.35,0.86,0.89,0.34,90.11,12.4,0.01,1.05,10.37,0.99,0.01,0.10,11.71,-0.00,0.01,0.13,11.26,0.88,1.16,0.16,91.68,14.3,0.01,1.08,15.57,0.01,0.15,17.25,-0.01,0.01,0.14,1.42,15.94


In [ ]:
# Grouping investment_class
# df.groupby("state")["last_name"].count()

result7.groupby("investment_class")["tag"].count()   # There class imbalance, but we continue for now(we will 
                                                     # come back here if we need to optimize the model)

investment_class
Blend     10298
Growth     6671
Value      6551
Name: tag, dtype: int64

In [ ]:
# Also replacing the categorical var with actual values

# df.replace(to_replace=['Mike', 'Tom', 'Perl'],
#           value= ['Michael', 'Thomas', 'Python'], 
#           inplace=True)

result7.replace(to_replace=['Blend', 'Growth', 'Value'],
           value= ['1', '2', '3'], 
          inplace=True)



In [ ]:
# creating dummy variables so these will be used as independent variables without imposing any kind of 
# ordering between the three labels of investment_class.

result7 = pd.get_dummies(result7, columns=['investment_class'])

In [ ]:
 result7.head(2)

,fund_id,tag,fund_ratio_net_annual_expense,pb_ratio,ps_ratio,mmc,pc_ratio,pe_ratio,total_assets,yield,greatstone_rating,inception_date,fund_size,return_ytd,bb_rating,below_b_rating,others_rating,maturity_bond,b_rating,a_rating,aaa_rating,aa_rating,bbb_rating,duration_bond,portfolio_communication_allocation,portfolio_financial_services,portfolio_industrials_allocation,portfolio_tech_allocation,portfolio_materials_basic_allocation,portfolio_energy_allocation,portfolio_consumer_defence_allocation,portfolio_healthcare_allocation,portfolio_property_allocation,portfolio_utils_allocation,portfolio_cyclical_consumer_allocation,2014_category_return,2012_return_category,years_up,2018_return_category,category_return_1year,cash_percent_of_portfolio,2011_return_category,ytd_return_fund,years_down,2014_return_fund,category_return_1month,2013_return_fund,fund_return_3months,ytd_return_category,2017_category_return,1_year_return_fund,2015_return_fund,portfolio_convertable,3_months_return_category,portfolio_others,2016_return_fund,stock_percent_of_portfolio,2016_return_category,2011_return_fund,2010_return_fund,fund_return_3years,2012_fund_return,2018_return_fund,2017_return_fund,category_ratio_net_annual_expense,category_return_2015,1_month_fund_return,bond_percentage_of_porfolio,portfolio_preferred,2010_return_category,2013_category_return,3yrs_treynor_ratio_fund,3_years_alpha_fund,3years_category_std,3yrs_sharpe_ratio_fund,3yrs_treynor_ratio_category,3_years_return_mean_annual_fund,fund_beta_3years,3years_fund_r_squared,3years_fund_std,category_beta_3years,3_years_alpha_category,3_years_return_mean_annual_category,3yrs_sharpe_ratio_category,3years_category_r_squared,3_years_return_category,category_r_squared_5years,5yrs_sharpe_ratio_fund,5_years_alpha_fund,5years_fund_r_squared,5years_fund_std,5yrs_sharpe_ratio_category,5_years_beta_fund,5yrs_treynor_ratio_fund,5_years_return_mean_annual_fund,5_years_return_mean_annual_category,5yrs_treynor_ratio_category,5_years_return_fund,5_years_alpha_category,5_years_beta_category,5years_category_std,5_years_return_category,10years_category_r_squared,10yrs_sharpe_ratio_fund,10_years_alpha_fund,10years_fund_r_squared,10years_fund_std,10yrs_sharpe_ratio_category,10_years_beta_fund,10yrs_treynor_ratio_fund,10_years_return_mean_annual_category,10yrs_treynor_ratio_category,10_years_return_fund,10_years_alpha_category,10_years_beta_category,10years_category_std,10_years_return_mean_annual_fund,10_years_return_category,investment_class_1,investment_class_2,investment_class_3
0,264614c6-5ac3-4146-ba26-1674b136cb40,67922,1.44,1.71,1.31,"19,857.41",5.91,14.51,1.185000e+07,5.57,NaN,2015-02-02,Large,20.19,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,1.0,-16.32,13.05,1.19,NaN,20.19,2.0,NaN,4.20,NaN,20.19,19.10,-5.78,18.40,NaN,0.0,19.10,0.0,16.14,98.81,27.30,NaN,NaN,4.24,NaN,-12.23,-3.31,1.75,-34.98,4.12,0.0,0.0,NaN,NaN,2.46,-7.10,0.18,0.26,0.05,0.45,1.20,54.83,16.25,0.01,-0.04,0.01,0.00,0.42,7.36,0.51,NaN,NaN,NaN,NaN,-0.00,NaN,NaN,NaN,-0.00,-0.04,0.00,-0.11,0.01,0.20,-4.25,0.49,NaN,NaN,NaN,NaN,0.01,NaN,NaN,0.01,0.21,0.00,0.06,0.01,0.13,NaN,14.30,0,0,1
1,f5ad58c2-fdea-4087-8678-e04744f89f90,134783,0.58,5.30,3.38,"72,347.03",15.95,18.88,1.397000e+10,0.42,3.0,2012-05-30,Large,16.79,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.78,9.77,9.97,35.51,2.86,0.38,5.88,14.41,2.67,0.39,17.38,10.0,15.34,5.0,-2.09,10.71,0.10,NaN,16.79,1.0,14.25,2.12,35.46,16.79,15.67,27.67,12.18,5.6,0.0,15.67,0.0,1.64,99.90,3.23,NaN,NaN,14.39,NaN,-2.62,26.39,1.06,3.60,2.33,0.0,0.0,NaN,33.92,12.2,0.07,0.13,1.06,0.13,1.19,1.07,88.46,12.26,0.01,0.01,0.01,0.01,0.84,15.35,0.86,0.89,0.34,90.11,12.4,0.01,1.05,10.37,0.99,0.01,0.10,11.71,-0.00,0.01,0.13,11.26,0.88,1.16,0.16,91.68,14.3,0.01,1.08,15.57,0.01,0.15,17.25,-0.01,0.01,0.14,1.42,15.94,0,1,0


In [ ]:

print (result7['inception_date'].dtype)

object


In [ ]:
# converting inceptioin_date to number of days till present date

import datetime
from datetime import date
from datetime import datetime

result7['inception_date'] = pd.to_datetime(result7['inception_date'])

result7['inception_date']= result7['inception_date'].map(lambda x: (datetime.today() - x).days)

result7 = result7.rename(columns = {"inception_date": "inception_date_till_today_in_days"})

In [ ]:
result7.head(2)

,fund_id,tag,fund_ratio_net_annual_expense,pb_ratio,ps_ratio,mmc,pc_ratio,pe_ratio,total_assets,yield,greatstone_rating,inception_date_till_today_in_days,fund_size,return_ytd,bb_rating,below_b_rating,others_rating,maturity_bond,b_rating,a_rating,aaa_rating,aa_rating,bbb_rating,duration_bond,portfolio_communication_allocation,portfolio_financial_services,portfolio_industrials_allocation,portfolio_tech_allocation,portfolio_materials_basic_allocation,portfolio_energy_allocation,portfolio_consumer_defence_allocation,portfolio_healthcare_allocation,portfolio_property_allocation,portfolio_utils_allocation,portfolio_cyclical_consumer_allocation,2014_category_return,2012_return_category,years_up,2018_return_category,category_return_1year,cash_percent_of_portfolio,2011_return_category,ytd_return_fund,years_down,2014_return_fund,category_return_1month,2013_return_fund,fund_return_3months,ytd_return_category,2017_category_return,1_year_return_fund,2015_return_fund,portfolio_convertable,3_months_return_category,portfolio_others,2016_return_fund,stock_percent_of_portfolio,2016_return_category,2011_return_fund,2010_return_fund,fund_return_3years,2012_fund_return,2018_return_fund,2017_return_fund,category_ratio_net_annual_expense,category_return_2015,1_month_fund_return,bond_percentage_of_porfolio,portfolio_preferred,2010_return_category,2013_category_return,3yrs_treynor_ratio_fund,3_years_alpha_fund,3years_category_std,3yrs_sharpe_ratio_fund,3yrs_treynor_ratio_category,3_years_return_mean_annual_fund,fund_beta_3years,3years_fund_r_squared,3years_fund_std,category_beta_3years,3_years_alpha_category,3_years_return_mean_annual_category,3yrs_sharpe_ratio_category,3years_category_r_squared,3_years_return_category,category_r_squared_5years,5yrs_sharpe_ratio_fund,5_years_alpha_fund,5years_fund_r_squared,5years_fund_std,5yrs_sharpe_ratio_category,5_years_beta_fund,5yrs_treynor_ratio_fund,5_years_return_mean_annual_fund,5_years_return_mean_annual_category,5yrs_treynor_ratio_category,5_years_return_fund,5_years_alpha_category,5_years_beta_category,5years_category_std,5_years_return_category,10years_category_r_squared,10yrs_sharpe_ratio_fund,10_years_alpha_fund,10years_fund_r_squared,10years_fund_std,10yrs_sharpe_ratio_category,10_years_beta_fund,10yrs_treynor_ratio_fund,10_years_return_mean_annual_category,10yrs_treynor_ratio_category,10_years_return_fund,10_years_alpha_category,10_years_beta_category,10years_category_std,10_years_return_mean_annual_fund,10_years_return_category,investment_class_1,investment_class_2,investment_class_3
0,264614c6-5ac3-4146-ba26-1674b136cb40,67922,1.44,1.71,1.31,"19,857.41",5.91,14.51,1.185000e+07,5.57,NaN,1918,Large,20.19,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,1.0,-16.32,13.05,1.19,NaN,20.19,2.0,NaN,4.20,NaN,20.19,19.10,-5.78,18.40,NaN,0.0,19.10,0.0,16.14,98.81,27.30,NaN,NaN,4.24,NaN,-12.23,-3.31,1.75,-34.98,4.12,0.0,0.0,NaN,NaN,2.46,-7.10,0.18,0.26,0.05,0.45,1.20,54.83,16.25,0.01,-0.04,0.01,0.00,0.42,7.36,0.51,NaN,NaN,NaN,NaN,-0.00,NaN,NaN,NaN,-0.00,-0.04,0.00,-0.11,0.01,0.20,-4.25,0.49,NaN,NaN,NaN,NaN,0.01,NaN,NaN,0.01,0.21,0.00,0.06,0.01,0.13,NaN,14.30,0,0,1
1,f5ad58c2-fdea-4087-8678-e04744f89f90,134783,0.58,5.30,3.38,"72,347.03",15.95,18.88,1.397000e+10,0.42,3.0,2896,Large,16.79,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.78,9.77,9.97,35.51,2.86,0.38,5.88,14.41,2.67,0.39,17.38,10.0,15.34,5.0,-2.09,10.71,0.10,NaN,16.79,1.0,14.25,2.12,35.46,16.79,15.67,27.67,12.18,5.6,0.0,15.67,0.0,1.64,99.90,3.23,NaN,NaN,14.39,NaN,-2.62,26.39,1.06,3.60,2.33,0.0,0.0,NaN,33.92,12.2,0.07,0.13,1.06,0.13,1.19,1.07,88.46,12.26,0.01,0.01,0.01,0.01,0.84,15.35,0.86,0.89,0.34,90.11,12.4,0.01,1.05,10.37,0.99,0.01,0.10,11.71,-0.00,0.01,0.13,11.26,0.88,1.16,0.16,91.68,14.3,0.01,1.08,15.57,0.01,0.15,17.25,-0.01,0.01,0.14,1.42,15.94,0,1,0


In [ ]:
result7.groupby("fund_size")["tag"].count() # There class imbalance, but we continue for now(we will come back
                                            # here if we need to optimize the model)

fund_size
Large     14173
Medium     6009
Small      3338
Name: tag, dtype: int64

In [ ]:
# Also replacing the categorical var with actual values

result7.replace(to_replace=['Large', 'Medium', 'Small'],
           value= ['1', '2', '3'], 
          inplace=True)


In [ ]:
# creating dummy variables so these will be used as independent variables without imposing any kind of 
# ordering between the three labels of investment_class.

result7 = pd.get_dummies(result7, columns=['fund_size'])

####  Splitting data between train/validation and test datasets

In [ ]:
# Reading this submission data to separate test data

dfsample_submission = pd.read_csv('/content/drive/My Drive/AIML/Hackathon/sample_submission.csv')


In [ ]:
dfsample_submission = dfsample_submission.drop(labels="greatstone_rating", axis = 1)

In [ ]:
# To be used as train and validation dataset
#  df1 = df1[~df1.index.isin(df2.index)]

X = result7[~result7.index.isin(dfsample_submission.index)]

In [ ]:
X.shape

(20000, 123)

In [ ]:
# To be used as test dataset

y = result7[result7.set_index(['fund_id']).index.isin(dfsample_submission.set_index(['fund_id']).index)]

In [ ]:
y.shape

(5000, 123)

In [ ]:
# re-order based on sample_submission dataset
y = y.set_index('fund_id')
y = y.reindex(index=dfsample_submission['fund_id'])
y = y.reset_index()

In [ ]:
y.head(2)

,fund_id,tag,fund_ratio_net_annual_expense,pb_ratio,ps_ratio,mmc,pc_ratio,pe_ratio,total_assets,yield,greatstone_rating,inception_date_till_today_in_days,return_ytd,bb_rating,below_b_rating,others_rating,maturity_bond,b_rating,a_rating,aaa_rating,aa_rating,bbb_rating,duration_bond,portfolio_communication_allocation,portfolio_financial_services,portfolio_industrials_allocation,portfolio_tech_allocation,portfolio_materials_basic_allocation,portfolio_energy_allocation,portfolio_consumer_defence_allocation,portfolio_healthcare_allocation,portfolio_property_allocation,portfolio_utils_allocation,portfolio_cyclical_consumer_allocation,2014_category_return,2012_return_category,years_up,2018_return_category,category_return_1year,cash_percent_of_portfolio,2011_return_category,ytd_return_fund,years_down,2014_return_fund,category_return_1month,2013_return_fund,fund_return_3months,ytd_return_category,2017_category_return,1_year_return_fund,2015_return_fund,portfolio_convertable,3_months_return_category,portfolio_others,2016_return_fund,stock_percent_of_portfolio,2016_return_category,2011_return_fund,2010_return_fund,fund_return_3years,2012_fund_return,2018_return_fund,2017_return_fund,category_ratio_net_annual_expense,category_return_2015,1_month_fund_return,bond_percentage_of_porfolio,portfolio_preferred,2010_return_category,2013_category_return,3yrs_treynor_ratio_fund,3_years_alpha_fund,3years_category_std,3yrs_sharpe_ratio_fund,3yrs_treynor_ratio_category,3_years_return_mean_annual_fund,fund_beta_3years,3years_fund_r_squared,3years_fund_std,category_beta_3years,3_years_alpha_category,3_years_return_mean_annual_category,3yrs_sharpe_ratio_category,3years_category_r_squared,3_years_return_category,category_r_squared_5years,5yrs_sharpe_ratio_fund,5_years_alpha_fund,5years_fund_r_squared,5years_fund_std,5yrs_sharpe_ratio_category,5_years_beta_fund,5yrs_treynor_ratio_fund,5_years_return_mean_annual_fund,5_years_return_mean_annual_category,5yrs_treynor_ratio_category,5_years_return_fund,5_years_alpha_category,5_years_beta_category,5years_category_std,5_years_return_category,10years_category_r_squared,10yrs_sharpe_ratio_fund,10_years_alpha_fund,10years_fund_r_squared,10years_fund_std,10yrs_sharpe_ratio_category,10_years_beta_fund,10yrs_treynor_ratio_fund,10_years_return_mean_annual_category,10yrs_treynor_ratio_category,10_years_return_fund,10_years_alpha_category,10_years_beta_category,10years_category_std,10_years_return_mean_annual_fund,10_years_return_category,investment_class_1,investment_class_2,investment_class_3,fund_size_1,fund_size_2,fund_size_3
0,85a6edf9-db47-490c-981a-168ce90434bb,87272,0.66,0.00,0.0,0,0,0,3.362000e+08,3.21,NaN,8488,2.71,0.0,0.0,1.63,13.58,0.0,40.79,19.25,21.9,16.44,6.49,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,10.23,7.57,18.0,0.74,4.29,0.00,10.20,2.71,3.0,9.90,1.38,-3.55,2.71,2.48,3.27,4.67,3.33,0.0,2.48,0.1,0.44,0.00,0.58,10.13,2.86,2.28,7.32,0.84,4.47,0.94,2.42,1.49,99.9,0.00,1.18,-5.66,1.18,-0.20,0.03,0.35,0.01,0.19,0.83,97.77,2.93,0.01,-0.01,0.00,0.00,0.95,1.90,0.92,1.02,0.19,96.73,2.66,0.01,0.87,3.16,0.29,0.00,0.03,3.55,-0.00,0.01,0.03,3.16,0.92,1.15,0.12,97.12,3.66,0.01,0.97,4.4,0.00,0.04,4.71,-0.01,0.01,0.04,0.39,4.22,1,0,0,0,1,0
1,654dac08-5e5d-4cf0-870e-3167087de7d2,160073,1.75,1.85,1.59,"23,043.98",6.63,17.92,3.330000e+09,1.61,NaN,5513,12.01,0.0,0.0,0.00,NaN,0.0,0.00,0.00,0.0,0.00,NaN,19.02,0.0,0.0,0.0,0.0,16.2,0.0,0.0,0.0,64.78,0.0,17.08,6.90,11.0,2.76,17.48,3.59,10.58,12.01,2.0,11.78,2.80,19.21,12.01,11.05,11.83,14.85,-15.41,0.0,11.05,0.0,10.53,95.03,13.75,5.83,12.64,9.41,12.48,0.10,13.87,1.15,-9.86,3.51,0.0,1.38,8.60,18.37,15.69,3.25,0.11,0.94,0.26,0.78,0.52,36.65,8.60,0.00,0.05,0.01,0.01,0.13,9.39,0.16,0.41,0.05,53.49,10.75,0.01,0.71,5.47,0.43,0.01,0.25,4.68,0.06,0.00,0.12,8.87,0.29,0.97,3.04,69.89,11.92,0.01,0.72,15.98,0.01,0.30,11.89,0.07,0.00,0.11,1.00,12.25,0,0,1,1,0,0


In [ ]:
X = X.drop(labels= "fund_id", axis = 1)
X = X.drop(labels= "tag", axis = 1)


In [ ]:
X.head(2)

,fund_ratio_net_annual_expense,pb_ratio,ps_ratio,mmc,pc_ratio,pe_ratio,total_assets,yield,greatstone_rating,inception_date_till_today_in_days,return_ytd,bb_rating,below_b_rating,others_rating,maturity_bond,b_rating,a_rating,aaa_rating,aa_rating,bbb_rating,duration_bond,portfolio_communication_allocation,portfolio_financial_services,portfolio_industrials_allocation,portfolio_tech_allocation,portfolio_materials_basic_allocation,portfolio_energy_allocation,portfolio_consumer_defence_allocation,portfolio_healthcare_allocation,portfolio_property_allocation,portfolio_utils_allocation,portfolio_cyclical_consumer_allocation,2014_category_return,2012_return_category,years_up,2018_return_category,category_return_1year,cash_percent_of_portfolio,2011_return_category,ytd_return_fund,years_down,2014_return_fund,category_return_1month,2013_return_fund,fund_return_3months,ytd_return_category,2017_category_return,1_year_return_fund,2015_return_fund,portfolio_convertable,3_months_return_category,portfolio_others,2016_return_fund,stock_percent_of_portfolio,2016_return_category,2011_return_fund,2010_return_fund,fund_return_3years,2012_fund_return,2018_return_fund,2017_return_fund,category_ratio_net_annual_expense,category_return_2015,1_month_fund_return,bond_percentage_of_porfolio,portfolio_preferred,2010_return_category,2013_category_return,3yrs_treynor_ratio_fund,3_years_alpha_fund,3years_category_std,3yrs_sharpe_ratio_fund,3yrs_treynor_ratio_category,3_years_return_mean_annual_fund,fund_beta_3years,3years_fund_r_squared,3years_fund_std,category_beta_3years,3_years_alpha_category,3_years_return_mean_annual_category,3yrs_sharpe_ratio_category,3years_category_r_squared,3_years_return_category,category_r_squared_5years,5yrs_sharpe_ratio_fund,5_years_alpha_fund,5years_fund_r_squared,5years_fund_std,5yrs_sharpe_ratio_category,5_years_beta_fund,5yrs_treynor_ratio_fund,5_years_return_mean_annual_fund,5_years_return_mean_annual_category,5yrs_treynor_ratio_category,5_years_return_fund,5_years_alpha_category,5_years_beta_category,5years_category_std,5_years_return_category,10years_category_r_squared,10yrs_sharpe_ratio_fund,10_years_alpha_fund,10years_fund_r_squared,10years_fund_std,10yrs_sharpe_ratio_category,10_years_beta_fund,10yrs_treynor_ratio_fund,10_years_return_mean_annual_category,10yrs_treynor_ratio_category,10_years_return_fund,10_years_alpha_category,10_years_beta_category,10years_category_std,10_years_return_mean_annual_fund,10_years_return_category,investment_class_1,investment_class_2,investment_class_3,fund_size_1,fund_size_2,fund_size_3
5000,1.38,1.85,1.38,"2,734.61",9.62,16.94,8.393400e+08,0.0,NaN,7622,12.30,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,17.28,20.82,14.78,7.64,3.66,5.04,11.19,4.48,2.97,12.14,3.79,15.46,14.0,-12.72,-0.03,0.00,-4.07,12.30,5.0,1.75,-2.31,33.51,12.30,13.34,12.28,0.12,-1.35,0.0,13.34,0.0,20.19,100.00,20.78,-4.76,26.07,7.61,10.38,-12.03,6.11,1.15,-5.38,-2.35,0.0,0.0,25.61,37.39,5.26,-7.26,0.16,0.48,0.08,0.71,1.20,74.23,14.96,0.01,-0.05,0.01,0.01,0.68,10.17,0.65,0.32,-6.53,70.10,14.84,0.00,1.11,3.44,0.47,0.01,0.04,4.63,-0.06,0.01,0.15,5.62,0.76,0.75,-5.66,79.60,17.51,0.01,1.23,9.94,0.01,0.12,12.69,-0.03,0.01,0.17,1.13,14.67,0,1,0,0,0,1
5001,0.89,3.82,3.28,"5,530.82",14.4,20.18,1.640000e+09,0.0,4.0,3476,5.57,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,35.24,10.03,10.66,4.45,0.08,13.50,11.97,0.00,0.00,14.07,44.59,29.69,4.0,-14.88,-3.74,0.61,-35.50,5.57,4.0,63.80,9.68,-5.67,5.57,4.54,46.78,-2.38,1.12,0.0,4.54,0.0,-1.00,99.39,0.65,-36.35,NaN,10.06,31.74,-9.92,36.04,1.72,-2.89,8.18,0.0,0.0,NaN,-11.23,15.82,5.51,0.18,0.63,0.15,0.89,0.55,15.59,14.86,0.01,0.06,0.01,0.01,0.15,10.34,0.21,0.80,11.57,15.66,16.08,0.01,0.53,23.25,1.14,0.01,0.12,13.14,0.08,0.01,0.20,8.86,0.47,0.74,8.85,43.41,25.05,0.01,1.05,16.12,0.01,0.11,17.34,0.04,0.01,0.25,1.59,12.04,0,1,0,1,0,0


In [ ]:
X["fund_ratio_net_annual_expense"].dtypes

dtype('float64')

In [ ]:
X.reset_index(drop=True, inplace=True)

In [ ]:
X.shape

(20000, 121)

In [ ]:
# replace NAN with 0

X = X.apply(lambda x: x.fillna(0)) 

#### some insight into my result

In [ ]:
X.groupby(["greatstone_rating"]).count()  # the model will better predit 3.0 or 0.0

,fund_ratio_net_annual_expense,pb_ratio,ps_ratio,mmc,pc_ratio,pe_ratio,total_assets,yield,inception_date_till_today_in_days,return_ytd,bb_rating,below_b_rating,others_rating,maturity_bond,b_rating,a_rating,aaa_rating,aa_rating,bbb_rating,duration_bond,portfolio_communication_allocation,portfolio_financial_services,portfolio_industrials_allocation,portfolio_tech_allocation,portfolio_materials_basic_allocation,portfolio_energy_allocation,portfolio_consumer_defence_allocation,portfolio_healthcare_allocation,portfolio_property_allocation,portfolio_utils_allocation,portfolio_cyclical_consumer_allocation,2014_category_return,2012_return_category,years_up,2018_return_category,category_return_1year,cash_percent_of_portfolio,2011_return_category,ytd_return_fund,years_down,2014_return_fund,category_return_1month,2013_return_fund,fund_return_3months,ytd_return_category,2017_category_return,1_year_return_fund,2015_return_fund,portfolio_convertable,3_months_return_category,portfolio_others,2016_return_fund,stock_percent_of_portfolio,2016_return_category,2011_return_fund,2010_return_fund,fund_return_3years,2012_fund_return,2018_return_fund,2017_return_fund,category_ratio_net_annual_expense,category_return_2015,1_month_fund_return,bond_percentage_of_porfolio,portfolio_preferred,2010_return_category,2013_category_return,3yrs_treynor_ratio_fund,3_years_alpha_fund,3years_category_std,3yrs_sharpe_ratio_fund,3yrs_treynor_ratio_category,3_years_return_mean_annual_fund,fund_beta_3years,3years_fund_r_squared,3years_fund_std,category_beta_3years,3_years_alpha_category,3_years_return_mean_annual_category,3yrs_sharpe_ratio_category,3years_category_r_squared,3_years_return_category,category_r_squared_5years,5yrs_sharpe_ratio_fund,5_years_alpha_fund,5years_fund_r_squared,5years_fund_std,5yrs_sharpe_ratio_category,5_years_beta_fund,5yrs_treynor_ratio_fund,5_years_return_mean_annual_fund,5_years_return_mean_annual_category,5yrs_treynor_ratio_category,5_years_return_fund,5_years_alpha_category,5_years_beta_category,5years_category_std,5_years_return_category,10years_category_r_squared,10yrs_sharpe_ratio_fund,10_years_alpha_fund,10years_fund_r_squared,10years_fund_std,10yrs_sharpe_ratio_category,10_years_beta_fund,10yrs_treynor_ratio_fund,10_years_return_mean_annual_category,10yrs_treynor_ratio_category,10_years_return_fund,10_years_alpha_category,10_years_beta_category,10years_category_std,10_years_return_mean_annual_fund,10_years_return_category,investment_class_1,investment_class_2,investment_class_3,fund_size_1,fund_size_2,fund_size_3
greatstone_rating,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058,5058
1.0,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105
2.0,3375,3375,3375,3375,3375,3375,3375,3375,3375,3375,3375,3375,3375,3375,3375,3375,3375,3375,3375,33

In [ ]:
X["greatstone_rating"].dtypes

dtype('float64')

In [ ]:
# XX = XX.astype(str)
# XX.dtypes
#yy = yy.reshape((len(yy), 1))

####  Function for getting data 

In [ ]:


##def get_data():
    # generate dataset
XX = X.drop(labels= "greatstone_rating" , axis = 1).astype(str)
yy = X["greatstone_rating"]
    
    
    # prepare input data
oe = OrdinalEncoder()
oe.fit(XX)
XX = oe.transform(XX)
    
    # prepare target
encoder = LabelEncoder()
encoder.fit(yy)
encoded_yy = encoder.transform(yy)
    
dummy_yy = np_utils.to_categorical(encoded_yy)
    
    # split into train and validation
X_train, X_val, y_train, y_val = train_test_split(XX, dummy_yy, test_size=0.30, random_state=1)
    
    
##    return X_train, y_train, X_val, y_val
    
  



In [ ]:

##def get_model(X_train, y_train):
    # define model
from tensorflow.keras.layers import Activation, Dropout, Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from keras import optimizers
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(100, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(6, activation='softmax'))
    # compile model

model.compile(loss='categorical_crossentropy', optimizer='adam')
    # metrics=['accuracy']
    
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=20, verbose=1, mode='auto', baseline=None, restore_best_weights=True)

model.fit(X_train, y_train, validation_data=(X_val,y_val), callbacks=[monitor], batch_size=16)
 ##   return model

#model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=260, batch_size=10)



875/875 [==============================] - 2s 2ms/step - loss: 67.3715 - val_loss: 16.1515


In [ ]:
pred = model.predict(X_val)
predict_classes = np.argmax(pred,axis=1)
expected_classes = np.argmax(y_val,axis=1)
correct = accuracy_score(expected_classes,predict_classes)
print(f"Accuracy: {correct}")

Accuracy: 0.2865


####  Data Generation

In [ ]:
# generate data
##X_train, y_train, X_val, y_val = get_data()


# fit model
##model = get_model(X_train, y_train)

In [ ]:
#model.summary()

In [ ]:
# predict probabilities for Val set
##yhat_probs = model.predict(X_val, verbose=0)
# predict crisp classes for X_val set
##yhat_classes = model.predict_classes(X_val, verbose=0)

#### Let see Precision of our model

In [ ]:
from sklearn.metrics import precision_score
#from Ipython.display import Image
#from Ipython.core.display import HTML

In [ ]:
##yyy = X["greatstone_rating"]
##yyy = yyy[0:6000,]
##yyy.shape

In [ ]:
acct_bal = pd.Series(expected_classes, name='Actual')

In [ ]:
pred_bal = pd.Series(predict_classes, name='Predicted')

In [ ]:
# This is the precision of the individual labels
precision_score(acct_bal, pred_bal, average=None)

array([0.5       , 0.25      , 0.17391304, 0.27537452, 0.        ,
       0.        ])

In [ ]:
precision_score(acct_bal, pred_bal, average='macro')

0.1998812599254993

In [ ]:
# since we have class imbalance in our label values this is the one we will take
precision_score(acct_bal, pred_bal, average='micro')

0.2758333333333333

#### Saving and finaling the prediction

In [ ]:
y.head(2)

,fund_id,tag,fund_ratio_net_annual_expense,pb_ratio,ps_ratio,mmc,pc_ratio,pe_ratio,total_assets,yield,greatstone_rating,inception_date_till_today_in_days,return_ytd,bb_rating,below_b_rating,others_rating,maturity_bond,b_rating,a_rating,aaa_rating,aa_rating,bbb_rating,duration_bond,portfolio_communication_allocation,portfolio_financial_services,portfolio_industrials_allocation,portfolio_tech_allocation,portfolio_materials_basic_allocation,portfolio_energy_allocation,portfolio_consumer_defence_allocation,portfolio_healthcare_allocation,portfolio_property_allocation,portfolio_utils_allocation,portfolio_cyclical_consumer_allocation,2014_category_return,2012_return_category,years_up,2018_return_category,category_return_1year,cash_percent_of_portfolio,2011_return_category,ytd_return_fund,years_down,2014_return_fund,category_return_1month,2013_return_fund,fund_return_3months,ytd_return_category,2017_category_return,1_year_return_fund,2015_return_fund,portfolio_convertable,3_months_return_category,portfolio_others,2016_return_fund,stock_percent_of_portfolio,2016_return_category,2011_return_fund,2010_return_fund,fund_return_3years,2012_fund_return,2018_return_fund,2017_return_fund,category_ratio_net_annual_expense,category_return_2015,1_month_fund_return,bond_percentage_of_porfolio,portfolio_preferred,2010_return_category,2013_category_return,3yrs_treynor_ratio_fund,3_years_alpha_fund,3years_category_std,3yrs_sharpe_ratio_fund,3yrs_treynor_ratio_category,3_years_return_mean_annual_fund,fund_beta_3years,3years_fund_r_squared,3years_fund_std,category_beta_3years,3_years_alpha_category,3_years_return_mean_annual_category,3yrs_sharpe_ratio_category,3years_category_r_squared,3_years_return_category,category_r_squared_5years,5yrs_sharpe_ratio_fund,5_years_alpha_fund,5years_fund_r_squared,5years_fund_std,5yrs_sharpe_ratio_category,5_years_beta_fund,5yrs_treynor_ratio_fund,5_years_return_mean_annual_fund,5_years_return_mean_annual_category,5yrs_treynor_ratio_category,5_years_return_fund,5_years_alpha_category,5_years_beta_category,5years_category_std,5_years_return_category,10years_category_r_squared,10yrs_sharpe_ratio_fund,10_years_alpha_fund,10years_fund_r_squared,10years_fund_std,10yrs_sharpe_ratio_category,10_years_beta_fund,10yrs_treynor_ratio_fund,10_years_return_mean_annual_category,10yrs_treynor_ratio_category,10_years_return_fund,10_years_alpha_category,10_years_beta_category,10years_category_std,10_years_return_mean_annual_fund,10_years_return_category,investment_class_1,investment_class_2,investment_class_3,fund_size_1,fund_size_2,fund_size_3
0,85a6edf9-db47-490c-981a-168ce90434bb,87272,0.66,0.00,0.0,0,0,0,3.362000e+08,3.21,NaN,8488,2.71,0.0,0.0,1.63,13.58,0.0,40.79,19.25,21.9,16.44,6.49,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,10.23,7.57,18.0,0.74,4.29,0.00,10.20,2.71,3.0,9.90,1.38,-3.55,2.71,2.48,3.27,4.67,3.33,0.0,2.48,0.1,0.44,0.00,0.58,10.13,2.86,2.28,7.32,0.84,4.47,0.94,2.42,1.49,99.9,0.00,1.18,-5.66,1.18,-0.20,0.03,0.35,0.01,0.19,0.83,97.77,2.93,0.01,-0.01,0.00,0.00,0.95,1.90,0.92,1.02,0.19,96.73,2.66,0.01,0.87,3.16,0.29,0.00,0.03,3.55,-0.00,0.01,0.03,3.16,0.92,1.15,0.12,97.12,3.66,0.01,0.97,4.4,0.00,0.04,4.71,-0.01,0.01,0.04,0.39,4.22,1,0,0,0,1,0
1,654dac08-5e5d-4cf0-870e-3167087de7d2,160073,1.75,1.85,1.59,"23,043.98",6.63,17.92,3.330000e+09,1.61,NaN,5513,12.01,0.0,0.0,0.00,NaN,0.0,0.00,0.00,0.0,0.00,NaN,19.02,0.0,0.0,0.0,0.0,16.2,0.0,0.0,0.0,64.78,0.0,17.08,6.90,11.0,2.76,17.48,3.59,10.58,12.01,2.0,11.78,2.80,19.21,12.01,11.05,11.83,14.85,-15.41,0.0,11.05,0.0,10.53,95.03,13.75,5.83,12.64,9.41,12.48,0.10,13.87,1.15,-9.86,3.51,0.0,1.38,8.60,18.37,15.69,3.25,0.11,0.94,0.26,0.78,0.52,36.65,8.60,0.00,0.05,0.01,0.01,0.13,9.39,0.16,0.41,0.05,53.49,10.75,0.01,0.71,5.47,0.43,0.01,0.25,4.68,0.06,0.00,0.12,8.87,0.29,0.97,3.04,69.89,11.92,0.01,0.72,15.98,0.01,0.30,11.89,0.07,0.00,0.11,1.00,12.25,0,0,1,1,0,0


In [ ]:
y = y.drop(labels= "greatstone_rating", axis = 1)
y = y.drop(labels= "fund_id", axis = 1)
y = y.drop(labels= "tag", axis = 1)
y_test = y.astype(str)

# Encode and prepare input test data, y
oe = OrdinalEncoder()
oe.fit(y_test)
y_test = oe.transform(y_test)

In [ ]:
y_test.shape

(5000, 120)

In [ ]:
# predict probabilities for Val set
##y_test_probs = model.predict(y_test, verbose=0)

# predict crisp classes for X_val set

predy = model.predict(y_test)
y_test_classes = np.argmax(predy,axis=1)




In [ ]:
for i in range(len(y_test)):
  dfypred = pd.DataFrame(y_test_classes)
  #res.index = X_val.index # its important for comparison

dfypred.columns = ["greatstone_rating"]
#yres.to_csv("/content/drive/My Drive/AIML/Hackathon/prediction_results_greatstone_ratings.csv")

In [ ]:
#dfypred = pd.read_csv('/content/drive/My Drive/AIML/Hackathon/prediction_results_greatstone_ratings.csv')
dfypred.head()

,greatstone_rating
0,3
1,3
2,3
3,3
4,3


In [ ]:
#dfypred = dfypred.drop(labels= "Unnamed: 0", axis = 1)

In [ ]:
#dfsample_submission = dfsample_submission.drop(labels= "greatstone_rating", axis = 1)

In [ ]:
dfsample_submission.head()

,fund_id
0,85a6edf9-db47-490c-981a-168ce90434bb
1,654dac08-5e5d-4cf0-870e-3167087de7d2
2,75214019-f876-42a9-a316-328b759ac4ba
3,1fe8de6d-cd33-489e-a829-211f1f622eed
4,bd87a7a0-6ca5-4607-a0cf-d396ecd6037a


In [ ]:
# Joing the predicted result to the submission format containing the fund_id
rating = dfypred["greatstone_rating"]
dfsample_submission_file = dfsample_submission. join(rating)
dfsample_submission_file.head()

,fund_id,greatstone_rating
0,85a6edf9-db47-490c-981a-168ce90434bb,3
1,654dac08-5e5d-4cf0-870e-3167087de7d2,3
2,75214019-f876-42a9-a316-328b759ac4ba,3
3,1fe8de6d-cd33-489e-a829-211f1f622eed,3
4,bd87a7a0-6ca5-4607-a0cf-d396ecd6037a,3


In [ ]:
dfsample_submission_file.to_csv("/content/drive/My Drive/AIML/Hackathon/sample_submission_final.csv")

#### Deducing how many funds are hightly rated

In [ ]:
dfsample_submission_file.groupby(["greatstone_rating"]).count()

,fund_id
greatstone_rating,
0,44
1,16
2,20
3,4871
5,49


#### OBSERVATION:   
1. Since 5 is highly rated this will contain the best mutual funds to choose from

Lets find out which of them are these top rated funds

In [ ]:
# Reading the fund config file 
dffund_config = pd.read_csv('/content/drive/My Drive/AIML/Hackathon/fund_config.csv')

In [ ]:
dffund_config.head()

,category,parent_company,fund_id,fund_name
0,Energy Limited Partnership,DWS,264614c6-5ac3-4146-ba26-1674b136cb40,DWS RREEF MLP & Energy Infrastructure Fund - C...
1,Large Growth,Janus Henderson,f5ad58c2-fdea-4087-8678-e04744f89f90,Janus Henderson Research Fund Class N
2,Large Growth,Calvert Research and Management,3c13f4ab-02c4-4ca7-a133-7e996ec5d0c4,Calvert Equity Fund Class A
3,Large Value,Invesco,ff78bdd8-59eb-4cef-9f3c-b1baacce9554,Invesco Diversified Dividend Fund R5 Class
4,Target-Date 2035,PGIM Funds (Prudential),63d8406d-c525-494a-8e03-d4fc4cfcb571,Prudential Day One 2035 Fund Class R3


In [ ]:
# Joing fund_config and sample_submission files
dfsample_submission_file = dfsample_submission_file.astype(str)

FundsRec = pd.merge(dfsample_submission_file,
                 dffund_config,
                 on='fund_id', 
                 how='left')

# This is exactly 5000 examples see the head and tail below
FundsRec.head()

,fund_id,greatstone_rating,category,parent_company,fund_name
0,85a6edf9-db47-490c-981a-168ce90434bb,3,Muni Single State Long,Nuveen,Nuveen Kentucky Municipal Bond Fund Class I
1,654dac08-5e5d-4cf0-870e-3167087de7d2,3,Utilities,MFS,MFS Utilities Fund Class R1
2,75214019-f876-42a9-a316-328b759ac4ba,3,Nontraditional Bond,Metropolitan West Funds,Metropolitan West Unconstrained Bond Fund Class M
3,1fe8de6d-cd33-489e-a829-211f1f622eed,3,Target-Date 2020,PGIM Funds (Prudential),Prudential Day One 2020 Fund Class R1
4,bd87a7a0-6ca5-4607-a0cf-d396ecd6037a,3,Large Growth,PGIM Funds (Prudential),PGIM Jennison Growth Fund- Class C


In [ ]:
FundsRec.tail()

,fund_id,greatstone_rating,category,parent_company,fund_name
4995,cd0a6c98-3d60-4548-a9db-493d80702107,3,Small Growth,Victory,Victory RS Small Cap Equity Fund Class A
4996,5a824843-621b-469f-a7d3-bae5b78beece,3,Ultrashort Bond,Wells Fargo Funds,Wells Fargo Adjustable Rate Government Fund - ...
4997,acb8601f-bb42-4fe3-9823-c124d38cfc82,3,Options-based,Glenmede,Glenmede Secured Options Portfolio Institution...
4998,e3f00c88-98d9-4a44-930c-891513652f8a,3,High Yield Bond,Vanguard,Vanguard High-Yield Corporate Fund Investor Sh...
4999,ef6f6819-2879-41c3-9736-ab1bf4d97869,3,Pacific/Asia ex-Japan Stk,Matthews Asia Funds,Matthews Asia Innovators Fund Institutional Cl...


In [ ]:
# Lets sort

FundsRec.sort_values(by=['greatstone_rating'], inplace=True, ascending=False)

In [ ]:
pd.set_option('display.max_rows', 100)

#pd.reset_option('display.max_rows')

In [ ]:
FundsRec.head(100)

,fund_id,greatstone_rating,category,parent_company,fund_name
184,555def56-9050-4c1e-ae24-a43547c32083,5,Nontraditional Bond,JPMorgan,JPMorgan Unconstrained Debt Fund Class A Shares
4909,402747b9-2a25-4c29-b2c8-dcc7b8125a8d,5,Market Neutral,Cognios,Cognios Market Neutral Large Cap Fund Institut...
781,e7633441-e70f-4e16-8e1b-a95b3e0f7830,5,Inflation-Protected Bond,Vanguard,Vanguard Short-Term Inflation-Protected Securi...
743,a395211a-7bca-43e4-a88c-ce1e8d654fdf,5,Multialternative,Janus Henderson,Janus Henderson Diversified Alternatives Fund ...
4570,6a0c5461-c525-4d44-82d6-f3589d19cb02,5,Options-based,Nottingham,Arin Large Cap Theta Fund Institutional Class
871,c0fdbfda-821f-4308-83ca-6629dfbd340e,5,Emerging-Markets Local-Currency Bond,OppenheimerFunds,Oppenheimer Emerging Markets Local Debt Fund C...
84,2e945e82-376f-487b-adb8-402e0e04ca55,5,Emerging Markets Bond,Ashmore,Ashmore Emerging Markets Corporate Income Fund...
2001,9a6ea1d3-5dcd-4383-9bb4-42f14fb87d68,5,Inflation-Protected Bond,JPMorgan,JPMorgan Inflation Managed Bond Fund Class I S...
150,11380255-8c32-4277-8554-d485efb06f2c,5,Multialternative,AQR Funds,AQR Multi-Strategy Alternative Fund Class I
2073,b1d586d2-da95-4d3a-bc79-12b53fdcedfd,5,Inflation-Protected Bond,JPMorgan,JPMorgan Inflation Managed Bond Fund Class A S...
